In [1]:
from osgeo import gdal, gdal_array
import numpy as np
import os
from terracatalogueclient import Catalogue 
from shapely.geometry import Polygon
import geopandas as gpd
from shapely.geometry import box
import requests
from tqdm.auto import tqdm  # provides a progressbar
from pathlib import Path



In [ ]:
# create catalogue object and authenticate 
catalogue = Catalogue().authenticate() 

# search for products in the WorldCover collection 
products = catalogue.get_products("urn:eop:VITO:ESA_WorldCover_10m_2020_V1") 

# download the products to the given directory 
catalogue.download_products(products, "downloads") 

### Authenticate to the Terrascope platform (registration required) 
# create catalogue object and authenticate interactively with a browser 
# catalogue = Catalogue().authenticate()  

# or authenticate with username and password 
catalogue = catalogue.authenticate_non_interactive("username", "Password") 

### Filter catalogue 
# search for all products in the WorldCover collection 
products = catalogue.get_products("urn:eop:VITO:ESA_WorldCover_10m_2020_V1") 
products
# or filter to a desired geometry, by providing it as an argument to get_products 

bounds = (3, 33, 15, 45) 
geometry = Polygon.from_bounds(*bounds) 
#products = catalogue.get_products("urn:eop:VITO:ESA_WorldCover_10m_2020_V1", geometry=geometry) 

## Download 
# download the products to the given directory 
#catalogue.download_products(products, "downloads") 

In [2]:
# get AOI geometry (select a country name)
country = 'Somalia'

In [ ]:

s3_url_prefix = "https://esa-worldcover.s3.eu-central-1.amazonaws.com"

# load natural earth low res shapefile
ne = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))

geom = ne[ne.name == country].iloc[0].geometry

# Define the bounding box coordinates: [minx, miny, maxx, maxy]
bbox = [40.39407136, -1.92483662, 51.58562142, 13.63884234]

# Create a GeoDataFrame with a single geometry representing the bounding box
geom1 = gpd.GeoDataFrame(geometry=[box(bbox[0], bbox[1], bbox[2], bbox[3])], crs='EPSG:4326')
geom1= geom1.geometry.iloc[0]
#print(geom)
#geom = gpd.[40.39407136, -1.92483662, 51.58562142, 13.63884234].geometry

# load worldcover grid
url = f'{s3_url_prefix}/esa_worldcover_grid.geojson'
#print(url)
grid = gpd.read_file(url)
#grid = grid.reset_index(drop=True)
# get grid tiles intersecting AOI

# Reset indices of both GeoDataFrames
#grid = grid.reset_index(drop=True)
#geom = geom.reset_index(drop=True)
tiles = grid[grid.intersects(geom1)]
#tiles = gpd.overlay(grid, geom1, how='intersection')


In [ ]:

import matplotlib.pyplot as plt
# Plot both GeoDataFrames on the same figure
fig, ax = plt.subplots(figsize=(8, 8))

# Plot the grid GeoDataFrame
grid.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)

# Plot the bounding box GeoDataFrame
geom1.plot(ax=ax, color='red', alpha=0.5)

# Customize the plot as needed
ax.set_title("Overlay of GeoDataFrames")
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")

# Show the plot
plt.show()

In [3]:
# define the output folder path
output_folder = r"C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA"+f"/{country}"  # use current directory or set a different one to store downloaded files
# create the directory if it does not exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [ ]:

year = 2021  # setting this to 2020 will download the v100 product instead

# select version tag, based on the year
version = {2020: 'v100',
           2021: 'v200'}[year]

for tile in tqdm(tiles.ll_tile):
    url = f"{s3_url_prefix}/{version}/{year}/map/ESA_WorldCover_10m_{year}_{version}_{tile}_Map.tif"
    out_fn = Path(output_folder) / Path(url).name
    if not out_fn.exists():
        r = requests.get(url, allow_redirects=True)
        with open(out_fn, 'wb') as f:
            f.write(r.content)

In [4]:
# Set the working directory where your TIFF files are located
#working_directory = r"C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA"

# List all TIFF files in the working directory with complete file paths
tif_files = [os.path.join(output_folder, f) for f in os.listdir(output_folder) if f.endswith('.tif')]

for i in tif_files:
    print(i)


C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N00E039_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N00E042_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N00E045_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N03E039_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N03E042_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N03E045_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N03E048_Map.tif
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA/Somalia\ESA_WorldCover_10m_2021_v200_N06E039_

In [5]:
# Mosaic TIFF files into one single file
output_mosaic_path = r"C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA\mosaic"+f"\\{country}"
file_name="mosaic_warp"

mosaic_file = output_mosaic_path+"\\"+file_name+".vrt"
mosaic_file_tif = output_mosaic_path+"\\"+file_name+".tif"
# create the directory if it does not exist
if not os.path.exists(output_mosaic_path):
    os.makedirs(output_mosaic_path)

print(mosaic_file)
print(mosaic_file_tif)

C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA\mosaic\Somalia\mosaic_warp.vrt
C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA\mosaic\Somalia\mosaic_warp.tif


In [ ]:
#vrt_options = gdal.BuildVRTOptions(resampleAlg='nearest')
vrt_options = gdal.BuildVRTOptions(resampleAlg="nearest", addAlpha=True)
my_vrt = gdal.BuildVRT(mosaic_file,tif_files, options=vrt_options)
my_vrt = None
#my_vrt = gdal.Open(mosaic_file)



In [ ]:

# #translate_options = gdal.TranslateOptions(format="GTiff", bandList=[1], creationOptions="COMPRESS=LZW")

# mosaic_file_tif = output_mosaic_path+"/"+file_name+".tif"

# # Set the target spatial resolution
# #target_resolution =  0.0022457882102988
# xRes = 0.002245788210298799
# yRes = -0.0022457882102988

# ops= f"GTiff -ot Byte -b 1 -r mode -tr {xRes} {yRes}"
# # Use gdal.Translate to convert VRT to GeoTIFF
# gdal.Translate(mosaic_file_tif,mosaic_file, options= ops)#

# # Explicitly close the datasets
# #my_vrt = None
# mosaic_file_tif = None

# print("Conversion from VRT to GeoTIFF complete.")

In [ ]:
#output_mosaic_path = Path(r"C:\Users\oscar.bautista\OneDrive - World Food Programme\Downloads\WC_ESA\mosaic", country)
#mosaic_file_tif = Path(output_mosaic_path, f"{file_name}.tif")

In [7]:
# from pathlib import Path
# #download_folder = Path('C:/Users', username, 'Downloads')

# print(mosaic_file_tif)
# # Get the geotransform information of the reference TIFF
# ref_ds = gdal.Open(ref_tiff)
# # from osgeo import osr
# # proj = osr.SpatialReference()
# # proj.ImportFromEPSG(4326)
# # projection_wkt = proj.ExportToWkt()
# # ref_ds.SetProjection(projection_wkt)
# ref_geotransform = ref_ds.GetGeoTransform()
# ref_geotransform


# # Calculate the upper-left (ulx, uly) and lower-right (lrx, lry) coordinates
# ulx = ref_geotransform[0]
# uly = ref_geotransform[3]
# lrx = ulx + ref_ds.RasterXSize * ref_geotransform[1]
# lry = uly + ref_ds.RasterYSize * ref_geotransform[5]

# # Set options for gdal.Translate
# # options = f"-of GTiff -ot Byte -r mode -projWin [{roi[0]}, {roi[1]}, {roi[2]}, {roi[3]}]"
# options = f"GTiff -ot Byte -b 1 -r mode -projWin {ulx} {uly} {lrx} {lry}"

# kwargs = {
#     'format': 'GTiff',
#     'outputType': gdal.GDT_Byte,
#     #'bandList': [1],
#     #'outputBounds': [ulx, uly, lrx, lry],
#     'xRes':0.002245788210298799,
#     'yRes':-0.0022457882102988,
#     'width':4984,
#     'height':6932,
#     'projWin':[ulx,uly,lrx,lry],
#     'projWinSRS': 'EPSG:4326'
# }


# # Perform the reprojection and windowing
# gdal.Translate(str(mosaic_file_tif), mosaic_file, **kwargs)#options=options

# mosaic_file_tif = None


warp test

In [ ]:
# Define the reference raster
ref_tiff = r"C:\Geotar\SOM\geodata\Processed\Vegetation\season\ndvi_m_s.tif"

In [8]:
def get_extent(tif_file):
    dataset = gdal.Open(tif_file)
    if dataset is None:
        print("Failed to open the raster file.")
        return None

    # Get geotransform information
    geotransform = dataset.GetGeoTransform()
    if geotransform is None:
        print("Failed to get geotransform information.")
        return None

    # Extract extent
    minX = geotransform[0]
    maxY = geotransform[3]
    maxX = minX + geotransform[1] * dataset.RasterXSize
    minY = maxY + geotransform[5] * dataset.RasterYSize
    
    # Extract x and y resolutions
    xRes = abs(geotransform[1])
    yRes = abs(geotransform[5])

    return minX, minY, maxX, maxY, xRes, yRes

# Example usage
#tif_file = "your_raster_file.tif"
minX, minY, maxX, maxY, xRes, yRes = get_extent(ref_tiff)


In [9]:
kwargs={'format': 'GTiff',
        #'srcBands':[1],
        'outputBounds':[minX, minY, maxX, maxY],
        'outputBoundsSRS':'EPSG:4326',
        'xRes':xRes,
        'yRes':yRes,
        'width':4984,
        'height':6932,
}

gdal.Warp(mosaic_file_tif, mosaic_file, **kwargs)
mosaic_file_tif= None